In [458]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from riotwatcher import LolWatcher, ApiError
import time

### Related Doc.

- https://hextechdocs.dev/crawling-matches-using-the-riot-games-api/
- https://towardsdatascience.com/how-to-use-riot-api-with-python-b93be82dbbd6

In [4]:
api_key = 'RGAPI-d1d56f1a-022d-4ce6-a6dd-e4a4f43ce225'

watcher = LolWatcher(api_key)
my_region = 'na1'

me = watcher.summoner.by_name(my_region, 'LL Stylish')
print(me)

# Return the rank status for Doublelift
my_ranked_stats = watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

{'id': '5HAJ0hfWvrrquSAVwaWgRdlTCe8kmsV0Ker6UiNrFH59UQM', 'accountId': 'Z_s396xDAozKjjn5uu-Zo5lK15AS_3hyn4HUZ0aG47ce5GE', 'puuid': 'Mj97ukUjMXvsgpVdqJ189fzkIgf9hPAqZQ3YAdYkLTBjhBynkNkgceLYMoL4WpyFT4cUmMbpUU1VYw', 'name': 'LL Stylish', 'profileIconId': 5127, 'revisionDate': 1643252180000, 'summonerLevel': 220}
[{'leagueId': 'ca2c0b26-447a-45b0-a5f6-b6eb2f3be95c', 'queueType': 'RANKED_SOLO_5x5', 'tier': 'DIAMOND', 'rank': 'I', 'summonerId': '5HAJ0hfWvrrquSAVwaWgRdlTCe8kmsV0Ker6UiNrFH59UQM', 'summonerName': 'LL Stylish', 'leaguePoints': 21, 'wins': 22, 'losses': 28, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}]


## Get List of Challenger Players

In [5]:
# Get all challenger players in na ranked

na_cha = watcher.league.challenger_by_queue('na1', 'RANKED_SOLO_5x5')

players_count = len(na_cha['entries'])
player_id = []
player_name = []
player_puuid = []

for index, player in enumerate(na_cha['entries']):
    player_id.append(player['summonerId'])
    player_name.append(player['summonerName'])
    player_puuid.append(watcher.summoner.by_id('na1', player['summonerId'])['puuid'])
    if index % 20 == 19 :
        print(f'Progress: {index}/{players_count}')
        break

Progress: 19/223


## Collect Matche_IDs in Challenger Division

In [6]:
# Get all non-duplicated matches

chal_matches_ids = set()

for index, puuid in enumerate(player_puuid):
    chal_matches_ids.update(watcher.match.matchlist_by_puuid('americas'
                                                            ,puuid=puuid
                                                            ,count=100)
                          )
    if index%5==0: print(f'Progress : {index+1}/{len(player_puuid)}')
        
print(f'Number of matches : {len(chal_matches_ids)}')

Progress : 1/20
Progress : 6/20
Progress : 11/20
Progress : 16/20
Number of matches : 1811


## Get Each Match Information (including all players stats)

In [7]:
# Get all matches info

matches=[]

for index, match_id in enumerate(chal_matches_ids):
    matches.append(watcher.match.by_id('americas', match_id))
    if index%50==0: print(f'Progress : {index+1}/{len(chal_matches_ids)}')
    if index == 100: break

Progress : 1/1811
Progress : 51/1811
Progress : 101/1811


## Save Matches/Players Stats to .csv

In [ ]:
# Save matches data of each player in .csv file

df = pd.DataFrame()

for match in matches:
    game_version = match['info']['gameVersion']
    game_mode = match['info']['gameMode']
    game_duration = match['info']['gameDuration']
    
    if game_mode == 'CLASSIC':
        participants = pd.DataFrame(match['info']['participants'])
        participants.insert(0, 'matchId', match['metadata']['matchId'])
        participants['game_version'] = game_version
        participants['game_duration'] = game_duration
        df = df.append(pd.DataFrame(participants))

front_columns = ['matchId', 'summonerId', 'championName', 'teamId', 'teamPosition', 'kills', 'assists', 'deaths', 'goldEarned']
columns_ordered = front_columns + df.columns.drop(front_columns).to_list()
df = df.reindex(columns=columns_ordered)

df.to_csv('matches_data.csv', index=False)

## Get Match Timeline Data

In [456]:
# Get win/lose data of blue_team (teamId == 100) of each match id in .csv
# Select only matches with duration more than 15 minutes

df = pd.read_csv('matches_data.csv')
blue_wins = df.loc[(df['teamId']==100) & (df['game_duration'] >= 900)].pivot_table(values=['teamId', 'win'], index=['matchId'])
display(blue_wins.head())
print(blue_wins.shape)

,teamId,win
matchId,,
NA1_4134639110,100,False
NA1_4135794736,100,False
NA1_4136156707,100,False
NA1_4136537752,100,True
NA1_4137045009,100,True


(6081, 2)


In [469]:
matches_timeline = []
start = time.time()

for index, match_id in enumerate(blue_wins.index):
    matches_timeline.append(watcher.match.timeline_by_match(region='americas', match_id=match_id))
    
    if index%10==0:
        now = time.time()
        if index == 0:
            time_10_matchs = now - start
        else:
            time_10_matchs = now - prev
        prev = now
        print(f'Progress : {index}/{len(blue_wins.index)} Estimated Time Left : {(len(blue_wins.index)-index)/600*time_10_matchs:.2f} min(s)')

Progress : 0/6081 Estimated Time Left : 1055.98 min(s)
Progress : 10/6081 Estimated Time Left : 59.21 min(s)
Progress : 20/6081 Estimated Time Left : 55.41 min(s)
Progress : 30/6081 Estimated Time Left : 51.94 min(s)
Progress : 40/6081 Estimated Time Left : 57.49 min(s)
Progress : 50/6081 Estimated Time Left : 58.97 min(s)
Progress : 60/6081 Estimated Time Left : 61.67 min(s)
Progress : 70/6081 Estimated Time Left : 62.30 min(s)
Progress : 80/6081 Estimated Time Left : 53.47 min(s)
Progress : 90/6081 Estimated Time Left : 55.21 min(s)
Progress : 100/6081 Estimated Time Left : 706.96 min(s)
Progress : 110/6081 Estimated Time Left : 62.46 min(s)
Progress : 120/6081 Estimated Time Left : 54.66 min(s)
Progress : 130/6081 Estimated Time Left : 49.11 min(s)
Progress : 140/6081 Estimated Time Left : 49.75 min(s)
Progress : 150/6081 Estimated Time Left : 59.69 min(s)
Progress : 160/6081 Estimated Time Left : 54.74 min(s)
Progress : 170/6081 Estimated Time Left : 57.26 min(s)
Progress : 180/608

Progress : 1480/6081 Estimated Time Left : 38.11 min(s)
Progress : 1490/6081 Estimated Time Left : 40.56 min(s)
Progress : 1500/6081 Estimated Time Left : 556.62 min(s)
Progress : 1510/6081 Estimated Time Left : 38.22 min(s)
Progress : 1520/6081 Estimated Time Left : 43.89 min(s)
Progress : 1530/6081 Estimated Time Left : 48.10 min(s)
Progress : 1540/6081 Estimated Time Left : 39.70 min(s)
Progress : 1550/6081 Estimated Time Left : 40.77 min(s)
Progress : 1560/6081 Estimated Time Left : 41.07 min(s)
Progress : 1570/6081 Estimated Time Left : 42.65 min(s)
Progress : 1580/6081 Estimated Time Left : 48.32 min(s)
Progress : 1590/6081 Estimated Time Left : 36.38 min(s)
Progress : 1600/6081 Estimated Time Left : 534.40 min(s)
Progress : 1610/6081 Estimated Time Left : 40.86 min(s)
Progress : 1620/6081 Estimated Time Left : 43.83 min(s)
Progress : 1630/6081 Estimated Time Left : 42.58 min(s)
Progress : 1640/6081 Estimated Time Left : 45.07 min(s)
Progress : 1650/6081 Estimated Time Left : 44.

Progress : 2950/6081 Estimated Time Left : 32.87 min(s)
Progress : 2960/6081 Estimated Time Left : 28.82 min(s)
Progress : 2970/6081 Estimated Time Left : 31.64 min(s)
Progress : 2980/6081 Estimated Time Left : 35.06 min(s)
Progress : 2990/6081 Estimated Time Left : 29.11 min(s)
Progress : 3000/6081 Estimated Time Left : 346.61 min(s)
Progress : 3010/6081 Estimated Time Left : 34.26 min(s)
Progress : 3020/6081 Estimated Time Left : 31.35 min(s)
Progress : 3030/6081 Estimated Time Left : 30.89 min(s)
Progress : 3040/6081 Estimated Time Left : 32.89 min(s)
Progress : 3050/6081 Estimated Time Left : 28.17 min(s)
Progress : 3060/6081 Estimated Time Left : 32.59 min(s)
Progress : 3070/6081 Estimated Time Left : 34.06 min(s)
Progress : 3080/6081 Estimated Time Left : 33.23 min(s)
Progress : 3090/6081 Estimated Time Left : 33.04 min(s)
Progress : 3100/6081 Estimated Time Left : 319.14 min(s)
Progress : 3110/6081 Estimated Time Left : 29.24 min(s)
Progress : 3120/6081 Estimated Time Left : 27.

Progress : 4420/6081 Estimated Time Left : 16.45 min(s)
Progress : 4430/6081 Estimated Time Left : 15.74 min(s)
Progress : 4440/6081 Estimated Time Left : 16.83 min(s)
Progress : 4450/6081 Estimated Time Left : 14.84 min(s)
Progress : 4460/6081 Estimated Time Left : 13.42 min(s)
Progress : 4470/6081 Estimated Time Left : 14.95 min(s)
Progress : 4480/6081 Estimated Time Left : 14.95 min(s)
Progress : 4490/6081 Estimated Time Left : 17.27 min(s)
Progress : 4500/6081 Estimated Time Left : 186.69 min(s)
Progress : 4510/6081 Estimated Time Left : 15.31 min(s)
Progress : 4520/6081 Estimated Time Left : 13.62 min(s)
Progress : 4530/6081 Estimated Time Left : 14.02 min(s)
Progress : 4540/6081 Estimated Time Left : 13.04 min(s)
Progress : 4550/6081 Estimated Time Left : 21.69 min(s)
Progress : 4560/6081 Estimated Time Left : 15.56 min(s)
Progress : 4570/6081 Estimated Time Left : 14.75 min(s)
Progress : 4580/6081 Estimated Time Left : 13.00 min(s)
Progress : 4590/6081 Estimated Time Left : 14.0

Progress : 5900/6081 Estimated Time Left : 21.02 min(s)
Progress : 5910/6081 Estimated Time Left : 1.42 min(s)
Progress : 5920/6081 Estimated Time Left : 1.64 min(s)
Progress : 5930/6081 Estimated Time Left : 1.32 min(s)
Progress : 5940/6081 Estimated Time Left : 1.20 min(s)
Progress : 5950/6081 Estimated Time Left : 1.19 min(s)
Progress : 5960/6081 Estimated Time Left : 1.18 min(s)
Progress : 5970/6081 Estimated Time Left : 1.09 min(s)
Progress : 5980/6081 Estimated Time Left : 0.93 min(s)
Progress : 5990/6081 Estimated Time Left : 0.90 min(s)
Progress : 6000/6081 Estimated Time Left : 9.68 min(s)
Progress : 6010/6081 Estimated Time Left : 0.68 min(s)
Progress : 6020/6081 Estimated Time Left : 0.55 min(s)
Progress : 6030/6081 Estimated Time Left : 0.44 min(s)
Progress : 6040/6081 Estimated Time Left : 0.34 min(s)
Progress : 6050/6081 Estimated Time Left : 0.29 min(s)
Progress : 6060/6081 Estimated Time Left : 0.20 min(s)
Progress : 6070/6081 Estimated Time Left : 0.10 min(s)
Progress 

In [415]:
# Check types of events occured in typical LoL matches

event_types = set()

for match in matches_timeline:
    for timestep in match['info']['frames']:
        for event in timestep['events']:
            event_types.add(event['type'])
            if event['type'] == 'BUILDING_KILL':
                x = match
            
event_types     

{'BUILDING_KILL',
 'CHAMPION_KILL',
 'CHAMPION_SPECIAL_KILL',
 'CHAMPION_TRANSFORM',
 'DRAGON_SOUL_GIVEN',
 'ELITE_MONSTER_KILL',
 'GAME_END',
 'ITEM_DESTROYED',
 'ITEM_PURCHASED',
 'ITEM_SOLD',
 'ITEM_UNDO',
 'LEVEL_UP',
 'PAUSE_END',
 'SKILL_LEVEL_UP',
 'TURRET_PLATE_DESTROYED',
 'WARD_KILL',
 'WARD_PLACED'}

In [592]:
# Create .csv file with each team's stats at 10-min mark and whether the team proceeds to win or not
# For supervised learning -> Win prediction

data = pd.DataFrame()

for index, match in enumerate(matches_timeline):

    wards_placed = []
    wards_kill = []
    kills = []
    assists = []
    deaths = []

    dragon_kill = dict(red=0, blue=0)
    herald_kill = dict(red=0, blue=0)

    buildings_destroyed_blue = []
    buildings_destroyed_red = []


    # Retrieve every events in the match up until 10 minutes mark

    for timestamp in match['info']['frames'][:12]:
        for event in timestamp['events']:
            if event['type'] == 'WARD_PLACED' and event['wardType'] != 'UNDEFINED':
                wards_placed.append(event['creatorId'])
            elif event['type'] == 'WARD_KILL' and event['wardType'] != 'UNDEFINED':
                wards_kill.append(event['killerId'])
            elif event['type'] == 'CHAMPION_KILL':
                kills.append(event['killerId'])
                deaths.append(event['victimId'])
                try:
                    assists.extend(event['assistingParticipantIds'])
                except:
                    pass
            elif event['type'] == 'BUILDING_KILL':
                if event['teamId'] == 100:
                    buildings_destroyed_blue.append(event['buildingType'])
                else:
                    buildings_destroyed_red.append(event['buildingType'])

            elif event['type'] == 'ELITE_MONSTER_KILL':
                if event['monsterType'] == 'DRAGON':
                    if event['killerTeamId'] == 100:
                        dragon_kill['blue'] += 1
                    else:
                        dragon_kill['red'] += 1
                elif event['monsterType'] == 'RIFTHERALD':
                    if event['killerTeamId'] == 100:
                        herald_kill['blue'] += 1
                    else:
                        herald_kill['red'] += 1
                else:
                    see.append(event['monsterType'])
            else:
                pass


    # Get [total team gold, average team level,  total team exp, total cs] at 10 min marks

    total_gold = dict(red=0, blue=0)
    total_exp = dict(red=0, blue=0)
    total_cs = dict(red=0, blue=0)
    level = dict(red=[], blue=[])
    total_kills = dict(red=0, blue=0)
    total_assists = dict(red=0, blue=0)
    total_deaths = dict(red=0, blue=0)
    total_wards_placed = dict(red=0, blue=0)
    total_wards_kills = dict(red=0, blue=0)

    for player_id, info in match['info']['frames'][11]['participantFrames'].items():
        if int(player_id) <= 5:
            total_gold['blue'] += info['totalGold']
            total_exp['blue'] += info['xp']
            total_cs['blue'] += info['minionsKilled'] + info['jungleMinionsKilled']
            level['blue'].append(info['level'])
        else:
            total_gold['red'] += info['totalGold']
            total_exp['red'] += info['xp']
            total_cs['red'] += info['minionsKilled'] + info['jungleMinionsKilled']
            level['red'].append(info['level'])

    # Sum total team kills/assists/deaths

    for killer in kills:
        if killer <= 5:
            total_kills['blue'] += 1
        else:
            total_kills['red'] += 1

    for death_player in deaths:
        if death_player <= 5:
            total_deaths['blue'] += 1
        else:
            total_deaths['red'] += 1

    for assister in assists:
        if assister <= 5:
            total_assists['blue'] += 1
        else:
            total_assists['red'] += 1

    for ward_placer in wards_placed:
        if ward_placer <= 5:
            total_wards_placed['blue'] += 1
        else:
            total_wards_placed['red'] += 1

    for ward_killer in wards_kill:
        if ward_killer <= 5:
            total_wards_kills['blue'] += 1
        else:
            total_wards_kills['red'] += 1
    
    data = data.append(pd.DataFrame({'matchId': [match['metadata']['matchId']],
                                     'blue_wins': blue_wins.loc[match['metadata']['matchId']]['win'],
                                     'blue_kills': [total_kills['blue']],
                                     'blue_assists': [total_assists['blue']], 
                                     'blue_deaths': [total_deaths['blue']], 
                                     'blue_wards_placed': [total_wards_placed['blue']], 
                                     'blue_wards_kills': [total_wards_kills['blue']], 
                                     'blue_dragon_kills': [dragon_kill['blue']], 
                                     'blue_herald_kills': [herald_kill['blue']],
                                     'blue_tower_kills': [buildings_destroyed_red.count('TOWER_BUILDING')],
                                     'blue_inhibitor_kills': [buildings_destroyed_red.count('INHIBITOR_BUILDING')],
                                     'blue_total_gold': [total_gold['blue']],
                                     'blue_average_lvl': [np.mean(level['blue'])],
                                     'blue_total_exp': [total_exp['blue']],
                                     'blue_total_cs': [total_cs['blue']],
                                     'red_kills': [total_kills['red']],
                                     'red_assists': [total_assists['red']], 
                                     'red_deaths': [total_deaths['red']], 
                                     'red_wards_placed': [total_wards_placed['red']], 
                                     'red_wards_kills': [total_wards_kills['red']], 
                                     'red_dragon_kills': [dragon_kill['red']], 
                                     'red_herald_kills': [herald_kill['red']],
                                     'red_tower_kills': [buildings_destroyed_blue.count('TOWER_BUILDING')],
                                     'red_inhibitor_kills': [buildings_destroyed_blue.count('INHIBITOR_BUILDING')],
                                     'red_total_gold': [total_gold['red']],
                                     'red_average_lvl': [np.mean(level['red'])],
                                     'red_total_exp': [total_exp['red']],
                                     'red_total_cs': [total_cs['red']],
                                    }), ignore_index=True)
    
    if index%10==0:
        print(f'Progress : {index+1}/{len(matches_timeline)}')

data.to_csv('10mins_stats.csv', index=False)

Progress : 1/6081
Progress : 11/6081
Progress : 21/6081
Progress : 31/6081
Progress : 41/6081
Progress : 51/6081
Progress : 61/6081
Progress : 71/6081
Progress : 81/6081
Progress : 91/6081
Progress : 101/6081
Progress : 111/6081
Progress : 121/6081
Progress : 131/6081
Progress : 141/6081
Progress : 151/6081
Progress : 161/6081
Progress : 171/6081
Progress : 181/6081
Progress : 191/6081
Progress : 201/6081
Progress : 211/6081
Progress : 221/6081
Progress : 231/6081
Progress : 241/6081
Progress : 251/6081
Progress : 261/6081
Progress : 271/6081
Progress : 281/6081
Progress : 291/6081
Progress : 301/6081
Progress : 311/6081
Progress : 321/6081
Progress : 331/6081
Progress : 341/6081
Progress : 351/6081
Progress : 361/6081
Progress : 371/6081
Progress : 381/6081
Progress : 391/6081
Progress : 401/6081
Progress : 411/6081
Progress : 421/6081
Progress : 431/6081
Progress : 441/6081
Progress : 451/6081
Progress : 461/6081
Progress : 471/6081
Progress : 481/6081
Progress : 491/6081
Progress : 

Progress : 3991/6081
Progress : 4001/6081
Progress : 4011/6081
Progress : 4021/6081
Progress : 4031/6081
Progress : 4041/6081
Progress : 4051/6081
Progress : 4061/6081
Progress : 4071/6081
Progress : 4081/6081
Progress : 4091/6081
Progress : 4101/6081
Progress : 4111/6081
Progress : 4121/6081
Progress : 4131/6081
Progress : 4141/6081
Progress : 4151/6081
Progress : 4161/6081
Progress : 4171/6081
Progress : 4181/6081
Progress : 4191/6081
Progress : 4201/6081
Progress : 4211/6081
Progress : 4221/6081
Progress : 4231/6081
Progress : 4241/6081
Progress : 4251/6081
Progress : 4261/6081
Progress : 4271/6081
Progress : 4281/6081
Progress : 4291/6081
Progress : 4301/6081
Progress : 4311/6081
Progress : 4321/6081
Progress : 4331/6081
Progress : 4341/6081
Progress : 4351/6081
Progress : 4361/6081
Progress : 4371/6081
Progress : 4381/6081
Progress : 4391/6081
Progress : 4401/6081
Progress : 4411/6081
Progress : 4421/6081
Progress : 4431/6081
Progress : 4441/6081
Progress : 4451/6081
Progress : 44

PermissionError: [Errno 13] Permission denied: '10mins_stats.csv'